# (project title)

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import h5py
import os
import pickle

import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input, InputLayer, Dropout, BatchNormalization, Convolution2D, MaxPooling2D, GlobalMaxPool2D
from keras import activations, models, optimizers, losses
from keras.activations import relu
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, multilabel_confusion_matrix

## Data pre-processing

### Retrieving and analyzing the metadata

## MLP

In [4]:
mlp = Sequential()
mlp.add(Dense(200, activation='relu',input_shape=(4000, ))) # input layer  #4000 = sample rate 1000 * 4sec audio
mlp.add(Dropout(0.5))
mlp.add(BatchNormalization())
mlp.add(Dense(200,activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(BatchNormalization())
mlp.add(Dense(200,activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))    # output layer

mlp.compile(loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer='adam')

# summary
mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               800200    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 200)              800       
 hNormalization)                                        

## CNN

In [5]:
nclass = 10
inp = Input(shape=(40, 334, 1))        # MFCCs
norm_inp = BatchNormalization()(inp)
audio = Convolution2D(32, kernel_size=(3, 3), activation=activations.relu)(norm_inp)
audio = MaxPooling2D(pool_size=(2, 2))(audio)
audio = Dropout(rate=0.1)(audio)
audio = Convolution2D(64, kernel_size=(3, 3), activation=activations.relu)(audio)
audio = GlobalMaxPool2D()(audio)
audio = Dropout(rate=0.1)(audio)

dense_1 = BatchNormalization()(Dense(256, activation=activations.relu)(audio))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

cnn = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

cnn.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40, 334, 1)]      0         
                                                                 
 batch_normalization_2 (Batc  (None, 40, 334, 1)       4         
 hNormalization)                                                 
                                                                 
 conv2d (Conv2D)             (None, 38, 332, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 166, 32)      0         
 )                                                               
                                                                 
 dropout_3 (Dropout)         (None, 19, 166, 32)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 164, 64)       18496 

## main run

In [3]:
with open("../datasets/audio_df.pkl", 'rb') as f:
    audio_df = pickle.load(f)
    
with open("../datasets/mfcc_df.pkl", 'rb') as f:
    mfcc_df = pickle.load(f)

In [6]:
mlp_accuracies = []
cnn_accuracies = []
mlp_conf_matrices = []
cnn_conf_matrices = []
for i in range(10):
    tf.keras.backend.clear_session()
    train_df = audio_df.drop(audio_df[audio_df['fold'] == i+1].index)
    test_df = audio_df.drop(audio_df[audio_df['fold'] != i+1].index)

    X_train = np.array(train_df['audio'].tolist())
    X_test = np.array(test_df['audio'].tolist())
    y_train = np.array(train_df['label'].tolist())
    y_test = np.array(test_df['label'].tolist())
    
    # training the MLP
    n_epochs_mlp = 100
    n_batch_size_mlp = 32
    mlp.fit(X_train, y_train, batch_size=n_batch_size_mlp, epochs=n_epochs_mlp, validation_data=(X_test, y_test))
    y_pred = mlp.predict(X_test,n_batch_size_mlp)
    
    # performance metrics for MLP
    accuracy = mlp.evaluate(X_test,y_test,verbose=0)[1]
    mlp_accuracies.append(accuracy)
    y_pred = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)
    mlp_conf_matrix = confusion_matrix(y_test, y_pred)
    mlp_conf_matrices.append(mlp_conf_matrix)
    
    print(f"\nMLP - fold {i+1} done\n")
    
    # getting training and test sets containing the MFCCs
    train_df = mfcc_df.drop(mfcc_df[mfcc_df['fold'] == i+1].index)
    test_df = mfcc_df.drop(mfcc_df[mfcc_df['fold'] != i+1].index)
    
    X_train = np.array(train_df['mfcc'].tolist())
    X_test = np.array(test_df['mfcc'].tolist())
    y_train = np.array(train_df['label'].tolist())
    y_test = np.array(test_df['label'].tolist())
    
    # training the CNN
    n_epochs_cnn = 10
    n_batch_size_cnn = 32
    cnn.fit(X_train,y_train,epochs=n_epochs_cnn,batch_size=n_batch_size_cnn)
    y_pred = cnn.predict(X_test,n_batch_size_cnn)
    
    # performance metrics for CNN
    accuracy = cnn.evaluate(X_test,y_test,verbose=0)[1]
    cnn_accuracies.append(accuracy)
    y_pred = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)
    cnn_conf_matrix = confusion_matrix(y_test, y_pred)
    cnn_conf_matrices.append(cnn_conf_matrix)
    
    print(f"\nCNN - fold {i+1} done\n")

Epoch 1/100
246/246 [==============================] - 2s 6ms/step - loss: 2.5408 - accuracy: 0.1510 - val_loss: 2.2556 - val_accuracy: 0.1134
Epoch 2/100
246/246 [==============================] - 1s 5ms/step - loss: 2.2902 - accuracy: 0.1650 - val_loss: 2.2389 - val_accuracy: 0.1443
Epoch 3/100
246/246 [==============================] - 1s 5ms/step - loss: 2.1952 - accuracy: 0.1892 - val_loss: 2.2340 - val_accuracy: 0.1592
Epoch 4/100
246/246 [==============================] - 1s 5ms/step - loss: 2.1409 - accuracy: 0.1993 - val_loss: 2.2105 - val_accuracy: 0.1718
Epoch 5/100
246/246 [==============================] - 1s 6ms/step - loss: 2.1043 - accuracy: 0.2192 - val_loss: 2.2152 - val_accuracy: 0.1730
Epoch 6/100
246/246 [==============================] - 1s 4ms/step - loss: 2.0642 - accuracy: 0.2339 - val_loss: 2.2121 - val_accuracy: 0.1581
Epoch 7/100
246/246 [==============================] - 1s 5ms/step - loss: 2.0111 - accuracy: 0.2578 - val_loss: 2.2050 - val_accuracy: 0.1913